In [1]:
# 定义一个简单的层
from mxnet import nd, gluon
from mxnet.gluon import nn

class CenteredLayer(nn.Block):
    def __init__(self, **kwargs):
        super(CenteredLayer, self).__init__(**kwargs)
    
    def forward(self, x):
        return x - x.mean()

/home/ly/anaconda3/envs/learning/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
layer = CenteredLayer()
layer(nd.array([1,2,3,4,5]))


[-2. -1.  0.  1.  2.]
<NDArray 5 @cpu(0)>

In [7]:
net = nn.Sequential()
with net.name_scope():
    net.add(nn.Dense(128))
    net.add(nn.Dense(10))
    net.add(CenteredLayer())

In [8]:
net.initialize()
print(net)
y = net(nd.random.uniform(shape=(4, 11)))
y.mean()

Sequential(
  (0): Dense(None -> 128, linear)
  (1): Dense(None -> 10, linear)
  (2): CenteredLayer(
  
  )
)



[3.7252904e-10]
<NDArray 1 @cpu(0)>

In [9]:
# 带模型参数的自定义层
pd = gluon.ParameterDict(prefix='block1_')
pd.get('exciting_parameter_yay', shape=(3, 3))
pd

block1_ (
  Parameter block1_exciting_parameter_yay (shape=(3, 3), dtype=<class 'numpy.float32'>)
)

In [10]:
class MyDense(nn.Block):
    def __init__(self, units, in_units, **kwargs):
        super(MyDense, self).__init__(**kwargs)
        with self.name_scope():
            self.weight = self.params.get('weight', shape=(in_units, units))
            self.bias = self.params.get('bias', shape=(units,))
            
    def forward(self, x):
        linear = nd.dot(x, self.weight.data()) + self.bias.data()
        return nd.relu(linear)

In [11]:
dense = MyDense(5, in_units=10, prefix='o_my_dense_')
dense.params

o_my_dense_ (
  Parameter o_my_dense_weight (shape=(10, 5), dtype=<class 'numpy.float32'>)
  Parameter o_my_dense_bias (shape=(5,), dtype=<class 'numpy.float32'>)
)

In [12]:
dense.initialize()
dense(nd.random.uniform(shape=(2, 10)))


[[0.05496314 0.         0.19100279 0.         0.00830198]
 [0.05216216 0.00047007 0.19354886 0.         0.08527613]]
<NDArray 2x5 @cpu(0)>

In [13]:
net = nn.Sequential()
with net.name_scope():
    net.add(MyDense(32, in_units=64))
    net.add(MyDense(2, in_units=32))
net.initialize()
net(nd.random.uniform(shape=(2, 64)))


[[0.         0.06028574]
 [0.         0.05150148]]
<NDArray 2x2 @cpu(0)>

In [15]:
nn.Dense??